# Image Grid Showcase

Generate a 1xN comparison grid from a directory of images.

- First image is the **reference** (separated by extra space)
- Remaining images are **comparisons** (equidistant spacing)
- Works with **any number of images**
- Light grey background

In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import os

# Setup
current_dir = Path.cwd()
OUTPUT_DIR = current_dir.parent / "data/grids"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Setup complete!")
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
def load_images_from_directory(directory, max_images=None):
    """
    Load images from a directory.
    Supports: .png, .jpg, .jpeg, .webp
    
    Args:
        directory: Path to directory containing images
        max_images: Optional limit on number of images to load (None = all)
    """
    directory = Path(directory)
    extensions = {'.png', '.jpg', '.jpeg', '.webp'}
    
    image_files = sorted([
        f for f in directory.iterdir() 
        if f.suffix.lower() in extensions
    ])
    
    if max_images is not None:
        image_files = image_files[:max_images]
    
    images = []
    filenames = []
    for img_path in image_files:
        img = Image.open(img_path).convert('RGB')
        images.append(img)
        filenames.append(img_path.name)
        
    return images, filenames


def create_grid_with_matplotlib(images, filenames=None,
                                 gap_after_first=0.15,
                                 regular_gap=0.02,
                                 bg_color='#F5F5F5',
                                 figsize_per_image=4,
                                 show_labels=True,
                                 title=None):
    """
    Create grid using matplotlib for more control and display.
    Works with any number of images.
    
    Args:
        images: List of PIL Images (any length)
        filenames: Optional list of filenames for labels  
        gap_after_first: Relative width of gap after first image (0-1)
        regular_gap: Relative width of regular gaps (0-1)
        bg_color: Background color
        figsize_per_image: Width per image in inches
        show_labels: Show filename labels below images
        title: Optional title for the grid
    """
    n = len(images)
    if n == 0:
        print("No images to display")
        return None
    
    # Calculate figure width based on number of images
    figsize = (figsize_per_image * n + 2, figsize_per_image + 1)
    
    # Calculate width ratios
    # [img1, big_gap, img2, gap, img3, gap, ...]
    width_ratios = [1]  # First image
    if n > 1:
        width_ratios.append(gap_after_first)  # Big gap after reference
        for i in range(1, n):
            width_ratios.append(1)  # Image
            if i < n - 1:
                width_ratios.append(regular_gap)  # Small gap
    
    # Number of columns (images + gaps)
    ncols = len(width_ratios)
    
    fig, axes = plt.subplots(1, ncols, figsize=figsize,
                             gridspec_kw={'width_ratios': width_ratios})
    fig.patch.set_facecolor(bg_color)
    
    if ncols == 1:
        axes = [axes]
    
    # Track which axes get images
    img_idx = 0
    for i, ax in enumerate(axes):
        ax.set_facecolor(bg_color)
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Determine if this is an image slot or gap
        if n == 1:
            is_image_slot = (i == 0)
        else:
            # Pattern: [img, gap, img, gap, img, gap, ...]
            # After first image, alternates: big_gap, img, gap, img, gap...
            if i == 0:
                is_image_slot = True
            elif i == 1:
                is_image_slot = False  # Big gap
            else:
                is_image_slot = (i % 2 == 0)  # Even indices after gap are images
        
        if is_image_slot and img_idx < len(images):
            ax.imshow(images[img_idx])
            if show_labels and filenames and img_idx < len(filenames):
                # Truncate long filenames
                label = filenames[img_idx]
                if len(label) > 25:
                    label = label[:22] + '...'
                ax.set_xlabel(label, fontsize=9, color='#666666')
            
            # Add "Reference" label to first image
            if img_idx == 0:
                ax.set_title('Reference', fontsize=10, fontweight='bold', color='#333333')
            
            img_idx += 1
            for spine in ax.spines.values():
                spine.set_visible(False)
        else:
            # This is a gap - make it invisible
            ax.axis('off')
    
    if title:
        fig.suptitle(title, fontsize=14, fontweight='bold', y=1.02)
    
    plt.tight_layout()
    return fig

In [ ]:
# Configuration widgets

dir_input = widgets.Text(
    value='',
    description='Image Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px'),
    placeholder='Enter full path to directory with images'
)

max_images_input = widgets.IntText(
    value=0,
    description='Max images (0=all):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

gap_slider = widgets.FloatSlider(
    value=0.15,
    min=0.05,
    max=0.4,
    step=0.01,
    description='Gap after reference:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

regular_gap_slider = widgets.FloatSlider(
    value=0.02,
    min=0.0,
    max=0.1,
    step=0.005,
    description='Regular gap:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

show_labels_checkbox = widgets.Checkbox(
    value=True,
    description='Show filenames',
    style={'description_width': 'initial'}
)

title_input = widgets.Text(
    value='',
    description='Title (optional):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

generate_button = widgets.Button(
    description='Generate Grid',
    button_style='success',
    icon='image',
    layout=widgets.Layout(width='150px')
)

save_button = widgets.Button(
    description='Save Grid',
    button_style='primary',
    icon='save',
    layout=widgets.Layout(width='150px')
)

output_area = widgets.Output()

# Store current figure for saving
current_fig = None
current_dir_name = None

def on_generate(b):
    global current_fig, current_dir_name
    
    with output_area:
        output_area.clear_output(wait=True)
        
        directory = dir_input.value.strip()
        if not directory:
            print("Please enter a directory path")
            return
        
        directory = Path(directory)
        if not directory.exists():
            print(f"Directory not found: {directory}")
            return
        
        try:
            max_imgs = max_images_input.value if max_images_input.value > 0 else None
            images, filenames = load_images_from_directory(directory, max_images=max_imgs)
            
            if not images:
                print("No images found in directory")
                return
            
            print(f"Loaded {len(images)} images from {directory.name}")
            for fn in filenames:
                print(f"  - {fn}")
            print()
            
            title = title_input.value.strip() if title_input.value.strip() else None
            
            current_fig = create_grid_with_matplotlib(
                images,
                filenames=filenames if show_labels_checkbox.value else None,
                gap_after_first=gap_slider.value,
                regular_gap=regular_gap_slider.value,
                show_labels=show_labels_checkbox.value,
                title=title
            )
            current_dir_name = directory.name
            
            plt.show()
            
        except Exception as e:
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()

def on_save(b):
    global current_fig, current_dir_name
    
    with output_area:
        if current_fig is None:
            print("No grid to save. Generate a grid first.")
            return
        
        filename = f"grid_{current_dir_name}.png"
        filepath = OUTPUT_DIR / filename
        
        current_fig.savefig(filepath, dpi=150, bbox_inches='tight', 
                           facecolor=current_fig.get_facecolor())
        print(f"\nSaved: {filepath}")

generate_button.on_click(on_generate)
save_button.on_click(on_save)

# Display interface
interface = widgets.VBox([
    widgets.HTML("<h3>Image Grid Generator</h3>"),
    dir_input,
    max_images_input,
    widgets.HTML("<h4>Spacing</h4>"),
    gap_slider,
    regular_gap_slider,
    widgets.HTML("<h4>Options</h4>"),
    show_labels_checkbox,
    title_input,
    widgets.HTML("<hr>"),
    widgets.HBox([generate_button, save_button]),
    output_area
])

display(interface)

---

## Batch Processing

Generate grids for multiple directories at once.

In [ ]:
def generate_grid_for_directory(directory, output_dir, 
                                 gap_after_first=0.15,
                                 regular_gap=0.02,
                                 show_labels=True,
                                 max_images=None,
                                 min_images=2):
    """
    Generate and save a grid for a single directory.
    
    Args:
        directory: Path to directory with images
        output_dir: Where to save the grid
        gap_after_first: Gap after reference image
        regular_gap: Gap between other images
        show_labels: Show filename labels
        max_images: Optional limit on images (None = all)
        min_images: Minimum images required (default 2)
    """
    directory = Path(directory)
    output_dir = Path(output_dir)
    
    images, filenames = load_images_from_directory(directory, max_images=max_images)
    
    if len(images) < min_images:
        print(f"  Skipping {directory.name}: only {len(images)} images found (need {min_images}+)")
        return False
    
    fig = create_grid_with_matplotlib(
        images,
        filenames=filenames if show_labels else None,
        gap_after_first=gap_after_first,
        regular_gap=regular_gap,
        show_labels=show_labels,
        title=directory.name
    )
    
    output_dir.mkdir(parents=True, exist_ok=True)
    filename = f"grid_{directory.name}.png"
    filepath = output_dir / filename
    
    fig.savefig(filepath, dpi=150, bbox_inches='tight',
               facecolor=fig.get_facecolor())
    plt.close(fig)
    
    print(f"  Saved: {filename} ({len(images)} images)")
    return True


def batch_generate_grids(parent_dir, output_dir=None,
                         gap_after_first=0.15,
                         regular_gap=0.02,
                         show_labels=True,
                         max_images=None,
                         min_images=2):
    """
    Generate grids for all subdirectories containing images.
    
    Args:
        parent_dir: Directory containing subdirectories with images
        output_dir: Where to save grids (default: parent_dir/grids)
        gap_after_first: Gap after reference image
        regular_gap: Gap between other images
        show_labels: Show filename labels
        max_images: Optional limit on images per grid (None = all)
        min_images: Minimum images required per directory (default 2)
    """
    parent_dir = Path(parent_dir)
    
    if output_dir is None:
        output_dir = parent_dir / "grids"
    output_dir = Path(output_dir)
    
    print(f"Scanning: {parent_dir}")
    print(f"Output: {output_dir}")
    print(f"Min images: {min_images}, Max images: {max_images or 'unlimited'}")
    print()
    
    subdirs = [d for d in parent_dir.iterdir() if d.is_dir() and d.name != 'grids']
    
    success_count = 0
    for subdir in sorted(subdirs):
        print(f"Processing: {subdir.name}")
        try:
            if generate_grid_for_directory(subdir, output_dir,
                                           gap_after_first=gap_after_first,
                                           regular_gap=regular_gap,
                                           show_labels=show_labels,
                                           max_images=max_images,
                                           min_images=min_images):
                success_count += 1
        except Exception as e:
            print(f"  Error: {e}")
    
    print(f"\nGenerated {success_count} grids")

In [ ]:
# Example batch usage:
# batch_generate_grids(
#     parent_dir="/path/to/parent/containing/image/folders",
#     output_dir="/path/to/save/grids",  # optional, defaults to parent_dir/grids
#     gap_after_first=0.15,
#     regular_gap=0.02,
#     show_labels=True,
#     max_images=None,  # None = all images, or set a limit like 10
#     min_images=2      # Skip directories with fewer than this many images
# )